In [ ]:
#use an older version of scipy for sparse matrix operations
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

     |████████████████████████████████| 31.2MB 102kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
cuda = torch.device('cuda')     # Default CUDA device
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

#set random seed for reproducibility
torch.manual_seed(0)

#define the model 
inputsize = 3*32*32
hiddensizes = [192,192,192]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


In [ ]:

net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()

Sequential(
  (0): Linear(in_features=3072, out_features=192, bias=True)
  (1): ReLU()
  (2): Linear(in_features=192, out_features=192, bias=True)
  (3): ReLU()
  (4): Linear(in_features=192, out_features=192, bias=True)
  (5): ReLU()
  (6): Linear(in_features=192, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
#training   
epochs =10
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Epoch 0 - Training loss: 1.7298427282087028

 Training time in minutes =  0.37448320786158246
Epoch 1 - Training loss: 1.4865916729583155

 Training time in minutes =  0.6272601922353108
Epoch 2 - Training loss: 1.3781189025210603

 Training time in minutes =  0.8811095118522644
Epoch 3 - Training loss: 1.2977349370184457

 Training time in minutes =  1.132163405418396
Epoch 4 - Training loss: 1.2325722941809603

 Training time in minutes =  1.3828094363212586
Epoch 5 - Training loss: 1.1742671081781997

 Training time in minutes =  1.638085699081421
Epoch 6 - Training loss: 1.12185889467254

 Training time in minutes =  1.8895079453786214
Epoch 7 - Training loss: 1.072145133372158

 Training time in minutes =  2.141822858651479
Epoch 8 - Training loss: 1.0258834490843136

 Training time in minutes =  2.394305221239726
Epoch 9 - Training loss: 0.9789426107235881

 Training time in minutes =  2.6525800108909605


In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1, 3072)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
np.save('model_accuracy_5.npy',(correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.5299


In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*inputsize))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
dim_w

665472

In [ ]:
#w_init ,A run  
from scipy import sparse
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,131072*2,131072*4])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1032

 Dist from Wi = 0.025040787239740275 , Dist from Wf =11.249709555012528

 n_rows = 4 , Accuracy =0.103

 Dist from Wi = 0.029241238086806814 , Dist from Wf =11.249699421014023

 n_rows = 8 , Accuracy =0.103

 Dist from Wi = 0.044010379725458376 , Dist from Wf =11.249651336803066

 n_rows = 16 , Accuracy =0.1033

 Dist from Wi = 0.053607074781679746 , Dist from Wf =11.249609699659805

 n_rows = 32 , Accuracy =0.1036

 Dist from Wi = 0.07760664866959713 , Dist from Wf =11.249469735111955

 n_rows = 64 , Accuracy =0.103

 Dist from Wi = 0.10562952817516962 , Dist from Wf =11.249241508472236

 n_rows = 128 , Accuracy =0.1031

 Dist from Wi = 0.13889432519075717 , Dist from Wf =11.24887996555915

 n_rows = 256 , Accuracy =0.1035

 Dist from Wi = 0.20060132365422934 , Dist from Wf =11.247948756196179

 n_rows = 512 , Accuracy =0.1036

 Dist from Wi = 0.2950229159484292 , Dist from Wf =11.245868289832686

 n_rows = 1024 , Accuracy =0.

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#w_init ,A+I run  1
from scipy import sparse
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,131072*2,131072*4])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1031

 Dist from Wi = 0.010087933991364604 , Dist from Wf =11.249732901129692

 n_rows = 4 , Accuracy =0.1031

 Dist from Wi = 0.01328044724190901 , Dist from Wf =11.249729585323069

 n_rows = 8 , Accuracy =0.1034

 Dist from Wi = 0.04696109168370413 , Dist from Wf =11.249639406178554

 n_rows = 16 , Accuracy =0.1033

 Dist from Wi = 0.06405422447726301 , Dist from Wf =11.249555065401667

 n_rows = 32 , Accuracy =0.1031

 Dist from Wi = 0.09251476028566462 , Dist from Wf =11.249357009728083

 n_rows = 64 , Accuracy =0.1031

 Dist from Wi = 0.11992669137519883 , Dist from Wf =11.249098172827624

 n_rows = 128 , Accuracy =0.103

 Dist from Wi = 0.17612353268064757 , Dist from Wf =11.248358663130807

 n_rows = 256 , Accuracy =0.1026

 Dist from Wi = 0.22202222432199245 , Dist from Wf =11.247546321091294

 n_rows = 512 , Accuracy =0.1033

 Dist from Wi = 0.3173698650131495 , Dist from Wf =11.245259822801806

 n_rows = 1024 , Accuracy =0.

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#w_init ,I
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,131072*2,131072*4,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))

      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:] 


      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1031

 Dist from Wi = 0.017297044870277785 , Dist from Wf =11.249724126635492

 n_rows = 4 , Accuracy =0.103

 Dist from Wi = 0.02657783830904295 , Dist from Wf =11.249706028678116

 n_rows = 8 , Accuracy =0.1029

 Dist from Wi = 0.03592894811998221 , Dist from Wf =11.24968004984377

 n_rows = 16 , Accuracy =0.1023

 Dist from Wi = 0.07177234741534062 , Dist from Wf =11.249508471187195

 n_rows = 32 , Accuracy =0.1022

 Dist from Wi = 0.08557569837277312 , Dist from Wf =11.249411936319781

 n_rows = 64 , Accuracy =0.1014

 Dist from Wi = 0.10822718543229722 , Dist from Wf =11.24921681671659

 n_rows = 128 , Accuracy =0.1014

 Dist from Wi = 0.12441106952421539 , Dist from Wf =11.249049470908455

 n_rows = 256 , Accuracy =0.1015

 Dist from Wi = 0.18958839911359573 , Dist from Wf =11.248139772961904

 n_rows = 512 , Accuracy =0.1011

 Dist from Wi = 0.32110085523297394 , Dist from Wf =11.245153905302256

 n_rows = 1024 , Accuracy =0.1024

 Dist from Wi = 0.435

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
#w_init ,Irev
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,131072*2,131072*4,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))

      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:]  


      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1031

 Dist from Wi = 0.07638518085259417 , Dist from Wf =11.249478095330113

 n_rows = 4 , Accuracy =0.1031

 Dist from Wi = 0.1284332873540636 , Dist from Wf =11.249004267216959

 n_rows = 8 , Accuracy =0.1031

 Dist from Wi = 0.17305581310787704 , Dist from Wf =11.248406278167389

 n_rows = 16 , Accuracy =0.1031

 Dist from Wi = 0.2983484574812134 , Dist from Wf =11.245780555883371

 n_rows = 32 , Accuracy =0.1031

 Dist from Wi = 0.38072988262273766 , Dist from Wf =11.243292972685717

 n_rows = 64 , Accuracy =0.1031

 Dist from Wi = 0.5504849168292401 , Dist from Wf =11.236260875821166

 n_rows = 128 , Accuracy =0.1031

 Dist from Wi = 0.6778093343662761 , Dist from Wf =11.229299471445753

 n_rows = 256 , Accuracy =0.1053

 Dist from Wi = 1.0156108841650475 , Dist from Wf =11.203799652132817

 n_rows = 512 , Accuracy =0.1126

 Dist from Wi = 1.438509716539057 , Dist from Wf =11.157386876351858

 n_rows = 1024 , Accuracy =0.1175

 Dist from Wi = 2.00497855

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
#w_init ,A run  2
from scipy import sparse
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1012

 Dist from Wi = 0.04922981138516871 , Dist from Wf =26.770305429532474

 n_rows = 4 , Accuracy =0.1012

 Dist from Wi = 0.057699799894491574 , Dist from Wf =26.770288513534684

 n_rows = 8 , Accuracy =0.1008

 Dist from Wi = 0.07650078929271967 , Dist from Wf =26.770241388414796

 n_rows = 16 , Accuracy =0.1009

 Dist from Wi = 0.1240507273887736 , Dist from Wf =26.770063275641256

 n_rows = 32 , Accuracy =0.1006

 Dist from Wi = 0.18053671937720991 , Dist from Wf =26.769741927361903

 n_rows = 64 , Accuracy =0.1009

 Dist from Wi = 0.24615401835462156 , Dist from Wf =26.76921897560162

 n_rows = 128 , Accuracy =0.102

 Dist from Wi = 0.30676984868351054 , Dist from Wf =26.768592951895595

 n_rows = 256 , Accuracy =0.1029

 Dist from Wi = 0.5029523942106785 , Dist from Wf =26.765625627902445

 n_rows = 512 , Accuracy =0.1019

 Dist from Wi = 0.7371797732249097 , Dist from Wf =26.76019884732645

 n_rows = 1024 , Accuracy =0.1028

In [ ]:
np.save("accuracy_3.npy", acc)
np.save("distance_i_3.npy", dist_i)
np.save("distance_f_3.npy", dist_f)

In [ ]:
#w_init ,A+I run  1
from scipy import sparse
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1008

 Dist from Wi = 0.07202606814941494 , Dist from Wf =26.770253801751245

 n_rows = 4 , Accuracy =0.1007

 Dist from Wi = 0.08814223383218459 , Dist from Wf =26.770205589635097

 n_rows = 8 , Accuracy =0.1005

 Dist from Wi = 0.10930889554255784 , Dist from Wf =26.77012752920746

 n_rows = 16 , Accuracy =0.1007

 Dist from Wi = 0.14736102543025983 , Dist from Wf =26.76994510814297

 n_rows = 32 , Accuracy =0.101

 Dist from Wi = 0.1889130455290607 , Dist from Wf =26.769684126372795

 n_rows = 64 , Accuracy =0.1011

 Dist from Wi = 0.2699877744577083 , Dist from Wf =26.768989203288516

 n_rows = 128 , Accuracy =0.1007

 Dist from Wi = 0.37448358287013894 , Dist from Wf =26.767731289937107

 n_rows = 256 , Accuracy =0.1018

 Dist from Wi = 0.5038008277275886 , Dist from Wf =26.765609671562956

 n_rows = 512 , Accuracy =0.1012

 Dist from Wi = 0.733098286895426 , Dist from Wf =26.760310971015148

 n_rows = 1024 , Accuracy =0.1018

 

In [ ]:
#w_init ,A+I run  2
from scipy import sparse
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1008

 Dist from Wi = 0.053396960428605346 , Dist from Wf =26.867452017473422

 n_rows = 4 , Accuracy =0.1008

 Dist from Wi = 0.0737986380034422 , Dist from Wf =26.867403724731485

 n_rows = 8 , Accuracy =0.1008

 Dist from Wi = 0.0913852004273118 , Dist from Wf =26.867349662589014

 n_rows = 16 , Accuracy =0.1008

 Dist from Wi = 0.12475704891605448 , Dist from Wf =26.867215427457296

 n_rows = 32 , Accuracy =0.1008

 Dist from Wi = 0.1586805545197019 , Dist from Wf =26.86703648764356

 n_rows = 64 , Accuracy =0.1008

 Dist from Wi = 0.2218475352027503 , Dist from Wf =26.86658915490228

 n_rows = 128 , Accuracy =0.1009

 Dist from Wi = 0.33746398348848655 , Dist from Wf =26.865385670186654

 n_rows = 256 , Accuracy =0.1009

 Dist from Wi = 0.49148670197730354 , Dist from Wf =26.86300932453918

 n_rows = 512 , Accuracy =0.1008

 Dist from Wi = 0.7246855083729522 , Dist from Wf =26.857729987133258

 n_rows = 1024 , Accuracy =0.1009



In [ ]:
#w_init ,A+I run  3
from scipy import sparse
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1132

 Dist from Wi = 0.0623138228089756 , Dist from Wf =49.18016411917008

 n_rows = 4 , Accuracy =0.1129

 Dist from Wi = 0.15862045332206348 , Dist from Wf =49.17994779737894

 n_rows = 8 , Accuracy =0.1129

 Dist from Wi = 0.1913211361799175 , Dist from Wf =49.17983145582703

 n_rows = 16 , Accuracy =0.1132

 Dist from Wi = 0.3440221358302549 , Dist from Wf =49.17900034132011

 n_rows = 32 , Accuracy =0.1134

 Dist from Wi = 0.41648280363859486 , Dist from Wf =49.178440071620926

 n_rows = 64 , Accuracy =0.1129

 Dist from Wi = 0.5354963670469323 , Dist from Wf =49.17728814649771

 n_rows = 128 , Accuracy =0.1135

 Dist from Wi = 0.6586590635041576 , Dist from Wf =49.17579276474858

 n_rows = 256 , Accuracy =0.1137

 Dist from Wi = 0.9372322558004029 , Dist from Wf =49.17127231926698

 n_rows = 512 , Accuracy =0.1148

 Dist from Wi = 1.3644834295006207 , Dist from Wf =49.161271451868075

 n_rows = 1024 , Accuracy =0.1147

 Dist f

In [ ]:
np.save("accuracy_AI3.npy", acc)
np.save("distance_i_AI3.npy", dist_i)
np.save("distance_f_AI3.npy", dist_f)

In [ ]:
#w_init ,I run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.0956

 Dist from Wi = 0.0029610119910475578 , Dist from Wf =3.006687461435312

 n_rows = 4 , Accuracy =0.0956

 Dist from Wi = 0.0033784635396158984 , Dist from Wf =3.006687021345661

 n_rows = 8 , Accuracy =0.0956

 Dist from Wi = 0.0034442782871287017 , Dist from Wf =3.006686946672602


KeyboardInterrupt: ignored

In [ ]:
#w_init ,I run  2
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.1008

 Dist from Wi = 0.03206443949985338 , Dist from Wf =26.867485945252508

 n_rows = 4 , Accuracy =0.1008

 Dist from Wi = 0.049413084428626526 , Dist from Wf =26.867459639752113

 n_rows = 8 , Accuracy =0.1008

 Dist from Wi = 0.05772623063795483 , Dist from Wf =26.867443064589295

 n_rows = 16 , Accuracy =0.1008

 Dist from Wi = 0.09320580128143643 , Dist from Wf =26.867343408411532

 n_rows = 32 , Accuracy =0.1008

 Dist from Wi = 0.1131404478204863 , Dist from Wf =26.867266857386067

 n_rows = 64 , Accuracy =0.1008

 Dist from Wi = 0.14454415151738426 , Dist from Wf =26.867116260127148

 n_rows = 128 , Accuracy =0.1008

 Dist from Wi = 0.20628831372211004 , Dist from Wf =26.866713127554924

 n_rows = 256 , Accuracy =0.1008

 Dist from Wi = 0.3153825859748084 , Dist from Wf =26.865653965080302

 n_rows = 512 , Accuracy =0.1009

 Dist from Wi = 0.5323178142712649 , Dist from Wf =26.86223123439981

 n_rows = 1024 , Accuracy =0.1011

 Dist from Wi = 0.8653

In [ ]:
#w_init ,I run  3
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.113

 Dist from Wi = 0.06454239722806825 , Dist from Wf =49.18016124495816

 n_rows = 4 , Accuracy =0.113

 Dist from Wi = 0.07657489800138412 , Dist from Wf =49.180143981958004

 n_rows = 8 , Accuracy =0.113

 Dist from Wi = 0.1090055341590912 , Dist from Wf =49.18008279369466

 n_rows = 16 , Accuracy =0.1133

 Dist from Wi = 0.16506773683073075 , Dist from Wf =49.17992658029235

 n_rows = 32 , Accuracy =0.1131

 Dist from Wi = 0.2184296773848514 , Dist from Wf =49.17971852580309

 n_rows = 64 , Accuracy =0.1134

 Dist from Wi = 0.3286216272374126 , Dist from Wf =49.17910566112444

 n_rows = 128 , Accuracy =0.1136

 Dist from Wi = 0.4574106729670965 , Dist from Wf =49.17807642924403

 n_rows = 256 , Accuracy =0.1142

 Dist from Wi = 0.6538664525945378 , Dist from Wf =49.17585672326307

 n_rows = 512 , Accuracy =0.1152

 Dist from Wi = 0.957236609561261 , Dist from Wf =49.170886954382524

 n_rows = 1024 , Accuracy =0.1159

 Dist from Wi = 1.4057822537479174 ,

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
# I UI experiment

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

In [ ]:
cuda = torch.device('cuda')     # Default CUDA device
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


#define the model 
inputsize = 3*32*32
hiddensizes = [192,192,192]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


In [ ]:
#Rerun for randomizing w_init

net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()

Sequential(
  (0): Linear(in_features=3072, out_features=192, bias=True)
  (1): ReLU()
  (2): Linear(in_features=192, out_features=192, bias=True)
  (3): ReLU()
  (4): Linear(in_features=192, out_features=192, bias=True)
  (5): ReLU()
  (6): Linear(in_features=192, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
epochs =15
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Epoch 0 - Training loss: 1.729568028236594

 Training time in minutes =  1.6794936498006185
Epoch 1 - Training loss: 1.4875438987751446

 Training time in minutes =  1.94804048538208
Epoch 2 - Training loss: 1.3807168214217476

 Training time in minutes =  2.2137892723083494
Epoch 3 - Training loss: 1.2970955438168763

 Training time in minutes =  2.4807428240776064
Epoch 4 - Training loss: 1.2294137451959692

 Training time in minutes =  2.748499377568563
Epoch 5 - Training loss: 1.174589165564998

 Training time in minutes =  3.0164491812388103
Epoch 6 - Training loss: 1.1167794607789314

 Training time in minutes =  3.2832290530204773
Epoch 7 - Training loss: 1.0653135941156646

 Training time in minutes =  3.5509296894073485
Epoch 8 - Training loss: 1.0206031990630546

 Training time in minutes =  3.8153542478879294
Epoch 9 - Training loss: 0.9718879281407427

 Training time in minutes =  4.07975138425827
Epoch 10 - Training loss: 0.9270515909127872

 Training time in minutes =  4.

In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1, 3072)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
np.save('model_accuracy_5.npy',(correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.5371


In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*inputsize))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
import numpy as np
a = np.asarray([1,2,3,4,5,6,7,8,9])
a[0:3]
a[3]

4

In [ ]:
#w_init ,I run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*inputsize))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
w_fin = np.concatenate((wf1,wf2,wf3,wf4), axis = 1)
w_fin = w_fin.T

w_fin.ravel()
w_fin.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.0928

 Dist from Wi = 0.034018324232412286 , Dist from Wf =15.238689641012359

 n_rows = 4 , Accuracy =0.0928

 Dist from Wi = 0.04360854836517953 , Dist from Wf =15.238665214381161

 n_rows = 8 , Accuracy =0.0928

 Dist from Wi = 0.04509147059997629 , Dist from Wf =15.238660898542236

 n_rows = 16 , Accuracy =0.0928

 Dist from Wi = 0.05109884879731115 , Dist from Wf =15.238641938477745

 n_rows = 32 , Accuracy =0.0928

 Dist from Wi = 0.0664582814639002 , Dist from Wf =15.238582693882197

 n_rows = 64 , Accuracy =0.0928

 Dist from Wi = 0.09714746038749943 , Dist from Wf =15.238417949131588

 n_rows = 128 , Accuracy =0.093

 Dist from Wi = 0.13311412508878986 , Dist from Wf =15.238146207830898

 n_rows = 256 , Accuracy =0.0927

 Dist from Wi = 0.18772471092222484 , Dist from Wf =15.237571284637946

 n_rows = 512 , Accuracy =0.0929

 Dist from Wi = 0.2904823999737824 , Dist from Wf =15.23595875542554

 n_rows = 1024 , Accuracy =0.0928

 Dist from Wi = 0.4077

In [ ]:
np.save("accuracy_I1.npy", acc)
np.save("distance_i_I1.npy", dist_i)
np.save("distance_f_I1.npy", dist_f)

In [ ]:
from scipy import sparse
dim_w = 600000
I = sparse.identity(dim_w).tocsr()
print('1')
Irev = I[::-1]
print('2') 

1
2


In [ ]:
#w_init ,I run  3
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)


      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:]


      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.0984

 Dist from Wi = 0.01735137507790828 , Dist from Wf =14.92644020165281

 n_rows = 4 , Accuracy =0.0983

 Dist from Wi = 0.033926638493645765 , Dist from Wf =14.926411730450651

 n_rows = 8 , Accuracy =0.0982

 Dist from Wi = 0.05079438127429152 , Dist from Wf =14.926363860450582

 n_rows = 16 , Accuracy =0.0981

 Dist from Wi = 0.054430543970365904 , Dist from Wf =14.926351043695085

 n_rows = 32 , Accuracy =0.0982

 Dist from Wi = 0.06521031509759301 , Dist from Wf =14.926307841477046

 n_rows = 64 , Accuracy =0.0982

 Dist from Wi = 0.0860056979285305 , Dist from Wf =14.926202503773627

 n_rows = 128 , Accuracy =0.0985

 Dist from Wi = 0.13575448749574864 , Dist from Wf =14.925832937657416

 n_rows = 256 , Accuracy =0.0986

 Dist from Wi = 0.19171516732803137 , Dist from Wf =14.925219042223487

 n_rows = 512 , Accuracy =0.0989

 Dist from Wi = 0.2757547435050409 , Dist from Wf =14.923902890503165

 n_rows = 1024 , Accuracy =0.0977

 Dist from Wi = 0.38

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
a = np.asarray([1,2,3,4,5,6,78])
a[-2:]

array([ 6, 78])

In [ ]:
#w_init ,Irev run  3
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000,192*3074,192*192 + 192*3074, 192*192 + 192*192 + 192*3074, 1920+192*192 + 192*192 + 192*3074])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)


      w[0:-n_rows] = wf_vec[0:-n_rows]
      w[-n_rows:] = w_init[-n_rows:]


      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in testloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")



 n_rows = 2 , Accuracy =0.5371

 Dist from Wi = 14.92640171880835 , Dist from Wf =0.038077453819994306

 n_rows = 4 , Accuracy =0.5355

 Dist from Wi = 14.925599458362267 , Dist from Wf =0.159370550197444

 n_rows = 8 , Accuracy =0.5359

 Dist from Wi = 14.924850265797991 , Dist from Wf =0.21854680790133915

 n_rows = 16 , Accuracy =0.5341

 Dist from Wi = 14.922027166111429 , Dist from Wf =0.3633502684281011

 n_rows = 32 , Accuracy =0.5324

 Dist from Wi = 14.913145036817465 , Dist from Wf =0.6300978293732736

 n_rows = 64 , Accuracy =0.5288

 Dist from Wi = 14.905604028660218 , Dist from Wf =0.7885979359106449

 n_rows = 128 , Accuracy =0.5095

 Dist from Wi = 14.888426933315285 , Dist from Wf =1.0647354671777707

 n_rows = 256 , Accuracy =0.4978

 Dist from Wi = 14.859394543374746 , Dist from Wf =1.4132628800259068

 n_rows = 512 , Accuracy =0.4714

 Dist from Wi = 14.79595796996998 , Dist from Wf =1.96940242576679

 n_rows = 1024 , Accuracy =0.4066

 Dist from Wi = 14.68351185093

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)